# Исследование надежности заемщиков.

**Заказчик — кредитный отдел банка. 
Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
Входные данные от банка — статистика о платёжеспособности клиентов.**

### Шаг 1. Обзор данных

In [1]:
import pandas as pd 
# импортируем библиотеку pandas

In [2]:
data = pd.read_csv('/datasets/data.csv') # читаем датасет и записываем в переменную
data.info() # смотрим общую информацию о датасете
data.head() # выводим на экран первые 10 строк, чтобы изучить датасет

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21525 non-null  int64  

 1   days_employed     19351 non-null  float64

 2   dob_years         21525 non-null  int64  

 3   education         21525 non-null  object 

 4   education_id      21525 non-null  int64  

 5   family_status     21525 non-null  object 

 6   family_status_id  21525 non-null  int64  

 7   gender            21525 non-null  object 

 8   income_type       21525 non-null  object 

 9   debt              21525 non-null  int64  

 10  total_income      19351 non-null  float64

 11  purpose           21525 non-null  object 

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Обнаруженные проблемы:
1. Общий трудовой стаж days_employed во всех строках выражен отрицательными значениями. Также присутсвуют аномально высокие значения.
2. В столбце education используются строчные и заглавные буквы.
3. Есть пропущенные значения в столбцах days_employed и total_income, количество пропущенных значений совпадает.

### Шаг 2.1 Заполнение пропусков

In [3]:
display(data['total_income'].isna().sum()) # считаем количество пропусков
display((data['total_income'].isna().sum() / len(data['total_income'])) * 100) 
# считаем отношение пропусков к длине столбца
data['total_income'] = data['total_income'].fillna(data['total_income'].median()) 
#заменяем пропуски на медианные значения
data.info() 

2174

10.099883855981417

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21525 non-null  int64  

 1   days_employed     19351 non-null  float64

 2   dob_years         21525 non-null  int64  

 3   education         21525 non-null  object 

 4   education_id      21525 non-null  int64  

 5   family_status     21525 non-null  object 

 6   family_status_id  21525 non-null  int64  

 7   gender            21525 non-null  object 

 8   income_type       21525 non-null  object 

 9   debt              21525 non-null  int64  

 10  total_income      21525 non-null  float64

 11  purpose           21525 non-null  object 

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


Были обнаружены пропуски в столбце total_income. 
Количество пропусков в каждом составляет 2 174 или около 10% от общего количеста.
Возможно, пропуски образовались из-за того, что клиенты не заполнили эти данные о себе. 
Или не были занесены в базу оператором.
Заменили пропуски на медианные значения по столбцу. 
Использовать медианные данные наиболее корректно, потому что в медиане отсечены крайние значения и они не оказывают сильное влияние на весь датафрейм.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [4]:
data['days_employed'] = data['days_employed'].abs() 
# меняем отрицательные значения на положительные
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median()) 
# заменяем пропуски на медианные значения
data['days_employed'] = data['days_employed'].astype('int')
# переводим вещественный тип в целочисленный
data.loc[data['days_employed'] > 14600, 'days_employed'] = data['days_employed'].median()
# заменяем неадеватно большие значения медианными
data['children'] = data['children'].abs()
# меняем отрицательные значения на положительные
data['children'] = data['children'].replace(20, 2)
# сомневаемся, что сейчас есть семьи с 20-тью детьми и заменяем их на 2
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 21525 entries, 0 to 21524

Data columns (total 12 columns):

 #   Column            Non-Null Count  Dtype  

---  ------            --------------  -----  

 0   children          21525 non-null  int64  

 1   days_employed     21525 non-null  float64

 2   dob_years         21525 non-null  int64  

 3   education         21525 non-null  object 

 4   education_id      21525 non-null  int64  

 5   family_status     21525 non-null  object 

 6   family_status_id  21525 non-null  int64  

 7   gender            21525 non-null  object 

 8   income_type       21525 non-null  object 

 9   debt              21525 non-null  int64  

 10  total_income      21525 non-null  float64

 11  purpose           21525 non-null  object 

dtypes: float64(2), int64(5), object(5)

memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2194.0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В столбце days_employed трудовой стаж в днях был выражен отрицательным числом. 
Скорее всего, это произошло из-за некорректного переноса данных из анкеты в базу, 
возможно, в анкете перед значением стояло тире.
Данные были переведены в положительные числа и по аналогии со столбцом total_income 
пропуски были заменены на медианные значения, чтобы не искажать датафрейм, и переведены из вещественного типа в целочисленный.
Также обнаружены черезчур большие значения количества дней трудового стажа, которые невозможны в принципе.
Заменяем их на медианные значения.
Поскольку количество детей не может быть отрицательным, заменяем всё на положительные значения.
И вряд ли в современных семьях есть по 20 детей, скорее всего это опечатка, поэтому меняем на 2.

### Шаг 2.3. Изменение типов данных.

In [5]:
data['total_income'] = data['total_income'].astype('int') # переводим вещественный тип данных в целочисленный
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,2194.0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

In [6]:
display(data.duplicated().sum()) # подсчитываем количество дубликатов
data['education'] = data['education'].str.lower() # переводим значения в столбце в строчное написание
data['family_status'] = data['family_status'].str.lower() 
# переводим значения в столбце в строчное написание
display(data.duplicated().sum()) 
# подсчитываем количество дубликатов после приведения к единому нижнему регистру
data = data.drop_duplicates().reset_index(drop=True) # удаляем дубликаты и реиндексируем датафрейм
data.duplicated().sum() # снова посчитываем количество дубликатов



54

71

0

Изначально было обнаружено 54 дубликата, после приведения к единому нижнему регистру 
в столбцах количество дубликатов увеличилось до 71.
Все дубликаты были удалены.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [7]:
edu = data[['education', 'education_id']] # создаём датафрейм по образованию
edu = edu.drop_duplicates().reset_index(drop=True) 
# удаляем из датафрейма дубликаты и пересчитываем индексы
fam = data[['family_status', 'family_status_id']] # создаём датафрейм по семейному статусу
fam = fam.drop_duplicates().reset_index(drop=True)
# удаляем из датафрейма дубликаты и пересчитываем индексы
display(edu)
display(fam)
data = data.drop(['education', 'family_status'], axis=1) # удаляем столбцы из основого датафрейма
data.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.0,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.0,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.0,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.0,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,2194.0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [8]:
def income_cat(total_income):
    if 0 <= total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    if total_income >= 1000001:
        return 'A'
    
    """создаём функцию, которая относит клиентов к разной категории в зависимости от дохода"""

data['total_income_category'] = data['total_income'].apply(income_cat) 
# добавляем новый столбец с категорией дохода использую функцию выше
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.0,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.0,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.0,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.0,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,2194.0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [9]:
display(data['purpose'].value_counts()) # смотрим, какие цели кредита присутсвуют в датафрейме

def purp_cat(purpose):
    if 'свад' in purpose:
        return 'проведение свадьбы'
    if 'недв' in purpose or 'жил' in purpose:
        return 'операции с недвижимостью'
    if 'авт' in purpose:
        return 'операции с автомобилем'
    if 'образован' in purpose:
        return 'получение образования'
    
    """создаём функцию, которая относит клиентов к разным категориям в зависимости от цели кредита"""
    
data['purpose_category'] = data['purpose'].apply(purp_cat)
# добавляем новый столбец с категорией дохода использую функцию выше
data.tail()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
21449,1,4529.0,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21450,0,2194.0,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21451,1,2113.0,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21452,3,3112.0,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем
21453,2,1984.0,40,1,0,F,сотрудник,0,82047,на покупку автомобиля,C,операции с автомобилем


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [10]:
data_debt = data[data['debt'] == 1] # выделяем датафрейм с заёмщиками, не вернувшими вовремя кредит
display((data.pivot_table(index='children', values='debt', aggfunc='mean') * 100)
        .sort_values(by='debt', ascending=False))
# а смотрим процент не вовремя возвращённых в срок кредитов 
# в зависимости от количества детей у клиента.

,debt
children,
4,9.756098
2,9.492481
1,9.165808
3,8.181818
0,7.543822
5,0.000000


**Вывод:** 

как видно из исследования, клиенты с детьми чаще не возвращают кредит вовремя. Клиентов с 5 детьми очень мало и все они вернули кредит вовремя.

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
display((data.pivot_table(index='family_status_id', values='debt', aggfunc='mean') * 100)
        .sort_values(by='debt', ascending=False))
# смотрим процент не вовремя возвращённых кредитов в зависимости от семейного положения клиента

,debt
family_status_id,
4,9.750890
1,9.347145
0,7.545182
3,7.112971
2,6.569343


**Вывод:** 

исходя из исследования, хуже всего возвращают кредит вовремя клиенты, которые никогда не были в браке и клиенты, которые находятся в гражданском браке, лучше всех вдовцы и вдовы.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
display((data.pivot_table(index='total_income_category', values='debt', aggfunc='mean') * 100)
        .sort_values(by='debt', ascending=False))

# смотрим процент не вовремя возвращённых кредитов в зависимости от категории дохода клиента


,debt
total_income_category,
E,9.090909
C,8.491508
A,8.000000
B,7.062091
D,6.000000


**Вывод:** 

Больше всего вовремя кредит не возвращают клиенты со сверхвысокими доходами (хотя их очень мало в этом датасете и выборка может быть нерепрезентативна) и клиенты со средними доходами. Меньше всего не возвращают кредит вовремя клиенты с высокими доходами.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [13]:
credit_aim = data_debt.groupby('purpose_category')['debt'].count()
# разделяем клиентов, не вернувших кредит в срок, в зависимости от цели кредита
credit_aim_total = data.groupby('purpose_category')['debt'].count()
# разделяем всех клиентов, взявших кредит, на группы в зависимости от цели кредита
credit_aim_ratio = (credit_aim / credit_aim_total) * 100
# находим в процентах отношение числа клиентов не вернувших кредит от общего числа клиентов,
# разделённых на группы в зависимости от целей кредитования
display(credit_aim_ratio.sort_values(ascending=False))

display((data.pivot_table(index='purpose_category', values='debt', aggfunc='mean') * 100)
        .sort_values(by='debt', ascending=False))

# смотрим процент не вовремя возвращённых кредитов в зависимости от цели получения кредита

# для сравнения оставлены оба варианта решения

purpose_category
операции с автомобилем      9.359034
получение образования       9.220035
проведение свадьбы          8.003442
операции с недвижимостью    7.233373
Name: debt, dtype: float64

,debt
purpose_category,
операции с автомобилем,9.359034
получение образования,9.220035
проведение свадьбы,8.003442
операции с недвижимостью,7.233373


**Вывод:** 

Хуже всего возвращают кредит вовремя клиенты, 
которые берут деньги на операции, связанные с покупкой автомобиля, 
и на получение образование. Самые надежные заёмщики - это клиенты, 
которые берут деньги на операции с недвижимостью.

## Общий вывод:

Если суммировать результат исследования, то выше всего вероятность не вернуть кредит в срок у клиентов, которые:
- имеют детей
- находятся в гражданском браке либо никогда не состояли в браке
- имеют средний доход
- берут деньги на операции с автомобилем или для получение образования

Самыми благонадёжными являются клиенты, которые:
- не имеют детей
- овдовели, находятся в разводе или состоят в браке
- имеют высокий доход
- берут деньги на операции с недвижимостью

В целом, результаты выглядят непротиворечиво, их можно логически обосновать.

В рамках исследования:
1. открыли и изучили датасет
2. нашли артефакты и аномальные значения
3. исправили найденные проблемы в данных
4. подготовили данные для дальнейшей работы
5. обработали данные и получили информацию, на основании которой можно в дальнейшем строить гипотезы и принимать решения
6. сделали выводы на основе полученной информации.
Как видно из исследования, в датасете как в процессе предобработки, так и при непосредственной работе с данными, постоянно вылезают артефакты и ошибки в данных. Из чего можно сделать вывод, что к предобработке данных нужно подходить с осбой тщательностью, потому что никогда не знаешь какая ошибка и как может повлиять на ход исследованя.